# Dual identification of novel phage receptor-binding proteins based on protein domains and machine learning

# *Predict new sequences*

This notebook, together with the 'RBPdetect_protein_embeddings' notebook can be used to make predictions for protein sequences based on our domain-based, machine-learning-based and/or combined approach.

### Instructions

1. Prepare a FASTA file with the **protein** sequences you want to make predictions for.

2. Download HMMER (http://hmmer.org), unzip its contents on your computer and locate this folder (e.g. '/Users/Sally/hmmer-3.3.1'). You can put this folder anywhere you want on your computer, as long as you know where it is.

3. Install all the necessary Python packages (*Libraries* below, click the hyperlinks for more info): [Biopython](https://biopython.org/wiki/Download), [tqdm](https://github.com/tqdm/tqdm#latest-pypi-stable-release), [xgboost](https://xgboost.readthedocs.io/en/stable/install.html), [NumPy](https://numpy.org/install/), [Pandas](https://pandas.pydata.org/docs/getting_started/install.html) and [Matplotlib](https://matplotlib.org/stable/users/installing/index.html). Typically you can install these packages via conda or pip, as mentioned in the installation guides that are hyperlinked. Often, NumPy, Pandas and Matplotlib come preinstalled.

4. Go to [Google Colab](http://colab.research.google.com/) or [Kaggle](https://www.kaggle.com) to compute the necessary protein language embeddings, see details in the enumeration below. After computing, save the computed embeddings from Kaggle or Google Colab locally on your computer.
    - In Colab, go to File > Upload notebook and then choose the 'RBPdetect_protein_embeddings' notebook in this repository after having it saved on your computer. Then Connect to a runtime (upper right of the screen) and finally go to Runtime > Change runtime type > make sure a GPU is enabled.
    - On the Kaggle platform, first sign in or make an account. Then click the 'Create' button on the left and start a new notebook. On the next screen, do File > Import notebook and upload the 'RBPdetect_protein_embeddings' notebook from this repository.

5. Copy the FASTA file and computed embeddings to the data folder of this repository. In the data folder, the RBPdetect_XGBmodel.json and/or RBPdetect_phageRBPs.hmm and/or RBPdetect_XGBhmm.json should also be located.

6. Fill in the necessary file names in the second code block below.

7. (Optional) To verify if HMMER software is properly installed and functioning, you can run predictions with the *Domain-based approach* (first few code blocks below) with the *sequences.fasta* example file. This file contains 3 protein sequences that are should all be predicted as RBPs (i.e., true positives). If this is not the case, then HMMER is not functioning properly (for troubleshooting: try looking at the outputs of the Pfam database press step before making predictions -> do you see errors or anything unexpected?).

8. Run all the code blocks sequentially to make predictions based on the domain-based approach and/or machine-learning-based approach and/or combined approach (see different headers below). You can run code blocks either by clicking on them and pressing the play button 'Execute' on the top of the screen, or pressing shift+enter.

9. The resulting dataframe with predictions contains a row for each of the protein sequences that was submitted. A binary prediction (0/1) is made for each of the methods. A '0' indicates that the sequence is predicted not to be an RBP, while a '1' indicates that the sequence is predicted as an RBP.

/content/sample_data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install hmmer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 45.3 MB/s eta 0:00:00


In [3]:
!pip show hmmer

Name: hmmer
Version: 0.2.4
Summary: HMMER binaries.
Home-page: 
Author: Danilo Horta
Author-email: horta@ebi.ac.uk
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [4]:
!pip install Biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.3 MB/s eta 0:00:00


In [5]:
import sys
sys.path.append("/content/drive/MyDrive/PhageRBPdetection")


### Libraries and files

In [6]:
from Bio import SeqIO
from tqdm.notebook import tqdm
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import RBPdetect_utils as rbpu

In [7]:
pfam_file = '/content/drive/MyDrive/PhageRBPdetection/data/RBPdetect_phageRBPs.hmm'
xgb_model_embeddings = '/content/drive/MyDrive/PhageRBPdetection/data/RBPdetect_xgb_model.json'
xgb_model_combined = '/content/drive/MyDrive/PhageRBPdetection/data/RBPdetect_xgb_hmm.json'
fasta_file = '/content/drive/MyDrive/Phage_proteins.faa'
hmmer_path = '/content/drive/MyDrive/hmmer-3.4'
embeddings_file = '/content/drive/MyDrive/phage_protein_embeddingsHope.csv'

### Combined approach XGBoost + HMM scores

Here, we combine both the embeddings of the proteins and the scores of each protein against all of the collected HMMs. These two types of features are concatenated and finally predictions are made (N.B. as a result, here predictions are made with a different XGBoost model than before).

In [8]:
# define all the blocks we want scores for
new_blocks = ['Phage_T7_tail', 'Tail_spike_N', 'Prophage_tail', 'BppU_N', 'Mtd_N',
           'Head_binding', 'DUF3751', 'End_N_terminal', 'phage_tail_N', 'Prophage_tailD1',
           'DUF2163', 'Phage_fiber_2', 'unknown_N0', 'unknown_N1', 'unknown_N2', 'unknown_N3', 'unknown_N4',
            'unknown_N6', 'unknown_N10', 'unknown_N11', 'unknown_N12', 'unknown_N13', 'unknown_N17', 'unknown_N19',
            'unknown_N23', 'unknown_N24', 'unknown_N26','unknown_N29', 'unknown_N36', 'unknown_N45', 'unknown_N48',
            'unknown_N49', 'unknown_N53', 'unknown_N57', 'unknown_N60', 'unknown_N61', 'unknown_N65', 'unknown_N73',
            'unknown_N82', 'unknown_N83', 'unknown_N101', 'unknown_N114', 'unknown_N119', 'unknown_N122',
            'unknown_N163', 'unknown_N174', 'unknown_N192', 'unknown_N200', 'unknown_N206', 'unknown_N208',
            'Lipase_GDSL_2', 'Pectate_lyase_3', 'gp37_C', 'Beta_helix', 'Gp58', 'End_beta_propel',
            'End_tail_spike', 'End_beta_barrel', 'PhageP22-tail', 'Phage_spike_2',
            'gp12-short_mid', 'Collar',
            'unknown_C2', 'unknown_C3', 'unknown_C8', 'unknown_C15', 'unknown_C35', 'unknown_C54', 'unknown_C76',
            'unknown_C100', 'unknown_C105', 'unknown_C112', 'unknown_C123', 'unknown_C179', 'unknown_C201',
            'unknown_C203', 'unknown_C228', 'unknown_C234', 'unknown_C242', 'unknown_C258', 'unknown_C262',
            'unknown_C267', 'unknown_C268', 'unknown_C274', 'unknown_C286', 'unknown_C292', 'unknown_C294',
            'Peptidase_S74', 'Phage_fiber_C', 'S_tail_recep_bd', 'CBM_4_9', 'DUF1983', 'DUF3672']

In [9]:
# press the .hmm file for further use
output, err = rbpu.hmmpress_python(hmmer_path, pfam_file)

In [10]:
# get domains & scores
sequences = [str(sequence.seq) for sequence in SeqIO.parse(fasta_file, 'fasta')]
hmm_scores = {item:[0]*len(sequences) for item in new_blocks}
bar = tqdm(total=len(sequences), desc='Scanning the proteins', position=0, leave=True)
for i, sequence in enumerate(sequences):
    hits, scores, biases, ranges = rbpu.hmmscan_python_from_string(hmmer_path, pfam_file, sequence)
    for j, dom in enumerate(hits):
        hmm_scores[dom][i] = scores[j]
    bar.update(1)
bar.close()
hmm_scores_array = np.asarray(pd.DataFrame(hmm_scores))

Scanning the proteins:   0%|          | 0/395315 [00:00<?, ?it/s]

In [11]:
# load protein embeddings to make predictions for and concat them with the HMM scores
embeddings_df = pd.read_csv(embeddings_file)
embeddings = np.asarray(embeddings_df.iloc[:, 1:])
features = np.concatenate((embeddings, hmm_scores_array), axis=1)

# load trained model
xgb_saved = XGBClassifier()
xgb_saved.load_model(xgb_model_combined)

# make predictions with the XGBoost model
score_xgb = xgb_saved.predict_proba(features)[:,1]
preds_xgb = (score_xgb > 0.5)*1

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:09:08] WARNING: /workspace/src/learner.cc:873: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
  warnings.warn(smsg, UserWarning)


In [ ]:
# save predictions
names = [record.id for record in SeqIO.parse(fasta_file, 'fasta')]
sequences = [str(record.seq) for record in SeqIO.parse(fasta_file, 'fasta')]
preds_xgb = ['RBP' if x == 1 else 'nonRBP' for x in preds_xgb]
xgb_based_detections = pd.DataFrame({'name': names, 'ProteinSeq': sequences, 'scores': score_xgb, 'prediction': preds_xgb})
xgb_based_detections.to_csv('xgb_combined_detections.csv', index=False)

In [13]:
import pandas as pd


# Step 1: Read the CSV file into a DataFrame
df = pd.read_csv('/content/xgb_combined_detections.csv')

# Step 2: Filter rows with 'rbp' in the 'name' column
rbp_df = df[df['prediction']=='RBP']


# Step 3: Filter rows with protein sequences between 200 and 1500 amino acids
rbp_df = rbp_df[(rbp_df['ProteinSeq'].str.len() >= 200) & (rbp_df['ProteinSeq'].str.len() <= 1500)]


# Step 4: Save the filtered DataFrame to a new CSV file
rbp_df.to_csv('filtered_rbp.csv', index=False)